In [1]:
import numpy as np
import pandas as pd

from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

In [2]:
!ls data

adult.data.csv	android_devices.csv  user_device.csv  user_usage.csv


In [3]:
user_usage = pd.read_csv('data/user_usage.csv')
user_device = pd.read_csv('data/user_device.csv')
devices = pd.read_csv('data/android_devices.csv')

In [4]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [5]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [6]:
devices.head()

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


In [7]:
result_usage = pd.merge(user_usage, user_device[['use_id', 'platform', 'device']], on='use_id')
result_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [8]:
result = pd.merge(
    result_usage,
    devices[['Device', 'Retail Branding']],
    left_on='device',
    right_on='Device'
).drop(axis='columns', labels='Device')
result.rename(columns={'Retail Branding' : "manufacturer"}, inplace=True)

result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer
0,94.46,35.17,519.12,22790,android,D2303,Sony
1,99.23,35.58,519.12,22854,android,D2303,Sony
2,283.30,107.47,15573.33,22806,android,A0001,OnePlus
3,283.30,107.47,15573.33,22806,android,A0001,OnePlus
4,57.49,16.73,15573.33,22839,android,A0001,OnePlus


In [9]:
means = result.groupby('manufacturer').agg({
    "outgoing_mins_per_month": "mean",
    "outgoing_sms_per_month": "mean",
    "monthly_mb": "mean",
    "use_id": "count"
})

means

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
manufacturer,,,,
Lava,60.650000,261.900000,12458.670000,2
OnePlus,170.395000,62.100000,15573.330000,4
Sony,143.703846,39.114615,2715.352308,13


In [10]:
query1 = \
"""
SELECT * FROM user_usage
INNER JOIN
(SELECT use_id, platform, device FROM user_device) as usage
ON
user_usage.use_id=usage.use_id
"""

query2 = \
"""
SELECT * FROM sql_usage
INNER JOIN
(SELECT "Retail Branding" as manufacturer, Device as device FROM devices) as devices
ON
sql_usage.device=devices.device
"""

query3 = \
"""
SELECT
    manufacturer,
    AVG(outgoing_mins_per_month), AVG(outgoing_sms_per_month),
    AVG(monthly_mb), COUNT(use_id)
FROM sql_result GROUP BY manufacturer
"""

In [11]:
sql_usage = pysqldf(query1)
sql_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,use_id,platform,device
0,21.97,4.82,1557.33,22787,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,22790,android,D2303
4,71.59,79.26,1557.33,22792,22792,android,SM-G361F


In [12]:
sql_result = pysqldf(query2)
sql_result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,device
0,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
1,283.30,107.47,15573.33,22806,android,A0001,OnePlus,A0001
2,283.30,107.47,15573.33,22806,android,A0001,OnePlus,A0001
3,244.88,105.95,1557.33,22832,android,D5803,Sony,D5803
4,135.09,42.02,5191.12,22833,android,E6653,Sony,E6653


In [13]:
sql_means = pysqldf(query3)
sql_means

,manufacturer,AVG(outgoing_mins_per_month),AVG(outgoing_sms_per_month),AVG(monthly_mb),COUNT(use_id)
0,Lava,60.650000,261.900000,12458.670000,2
1,OnePlus,170.395000,62.100000,15573.330000,4
2,Sony,143.703846,39.114615,2715.352308,13


In [14]:
pandas_timings = []
sql_timings = []

In [15]:
%%timeit

pd.merge(user_usage, user_device[['use_id', 'platform', 'device']], on='use_id')

2.06 ms ± 20.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit

pysqldf(query1)

10.2 ms ± 595 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit

pd.merge(result,
    devices[['Device', 'Retail Branding']],
    left_on='device',
    right_on='Device'
).drop(axis='columns', labels='Device')
result.rename(columns={'Retail Branding' : "manufacturer"}, inplace=True)

4.04 ms ± 53.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit

pysqldf(query2)

87 ms ± 90.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit

result.groupby('manufacturer').agg({
    "outgoing_mins_per_month": "mean",
    "outgoing_sms_per_month": "mean",
    "monthly_mb": "mean",
    "use_id": "count"
})

1.43 ms ± 6.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
%%timeit

pysqldf(query3)

5.54 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
